In [4]:
import pandas as pd
import numpy as np
from os import walk

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/BOM T6'
file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)
dfs = (pd.read_excel(x, header=1) for x in file_names)
bom = pd.concat(dfs)
del bom['Unnamed: 0']

# Create level
bom['Level'] = bom['Level'].astype(str)
bom['Level'] = bom['Level'].str[-1]
bom['Level'] = pd.to_numeric(bom['Level'])

bom = bom[['Level', 'Component', 'Unit', 'Quantity']]

# bom['temp'] = bom['Level'].shift()
# bom['temp'].fillna(0, inplace=True)

# # RETURN TRUE IF ITEM HAS FATHER
# def let_find_your_father(row):
#     if row['Level'] > row['temp']:
#         return True

# bom['is_child'] = bom.apply (lambda row: let_find_your_father(row), axis=1)
# del bom['temp']


# bom['temp_material'] = bom['Component'].shift()
# def fill_father_name(row):
#     if row['is_child']:
#         return row['temp_material']

# bom['father'] = bom.apply (lambda row: fill_father_name(row), axis=1)
    
# def fill_finish_product_name(row):
#     if row['Level'] == 0:
#         return row['Component']

# bom['product'] = bom.apply (lambda row: fill_finish_product_name(row), axis=1)
# del bom['temp_material']
# # bom.ffill(inplace=True)
# def finish_up(row):
#     if row['Level'] == 0:
#         return None
#     else:
#         return row['father']

# bom['father'] = bom.apply (lambda row: finish_up(row), axis=1)
def find_if_item_is_child(row):
    if row['Level'] > 0:
        return True
bom['is_child'] = bom.apply(lambda row: find_if_item_is_child(row), axis=1)
def create_columns(row, i):
    if row['Level'] == i:
        return row['Component']
for i in bom['Level'].unique():
    bom[i] = bom.apply(lambda row: create_columns(row, i), axis=1)

bom.ffill(inplace=True)

def fill_father_name(row):
    if row['is_child'] and row['Level'] > 0:
        return row[row['Level'] - 1]

bom['father'] = bom.apply(lambda row: fill_father_name(row), axis=1)
bom['Product'] = bom[0]
for i in bom['Level'].unique():
    del bom[i]
bom.to_excel('../output/2.xlsx')
